##### 问题:
我们想将一个只读的属性定义为 property 属性方法，只有在访问它时才参与计算。但是，
一旦访问了该属性，我们希望把计算出的值缓存起来，不要每次访问它时都重新计算。

##### 解决方案:
定义一个惰性属性最有效的方式就是利用描述符类来完成，示例如下：

In [11]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            print(type(self),type(instance),type(self.func)) #这里的func应该是代表装饰器下面的函数
            value = self.func(instance) #我这里认为是回调Cirle中的函数,instance在这里是Circle类
            setattr(instance, self.func.__name__, value) #setattr() 函数对应函数 getattr()，用于设置属性值，该属性不一定是存在的。
            return value

import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2 
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    
c=Circle(4)
print(c.area)
print(c.perimeter)

print(c.area)
print(c.perimeter)

<class '__main__.lazyproperty'> <class '__main__.Circle'> <class 'function'>
Computing area
50.26548245743669
<class '__main__.lazyproperty'> <class '__main__.Circle'> <class 'function'>
Computing perimeter
25.132741228718345
50.26548245743669
25.132741228718345


请注意，这里的“Computing area”和“Computing perimeter”只打印了一次。

在大部分情况下，让属性具有惰性求值能力的全部意义就在于提升程序性能。例
如，除非确实需要用到这个属性，否则就可以避免进行无意义的计算。本节给出的
解决方案正是应对于此，而且利用了描述符的微妙特性，使得能够以高效的方式
来达成。

 8.9 节中讲过，当把描述符放到类的定义体中时，访问它的属性会触发_\_get__()、
\_\_set\_\_()和__delete__()方法得到执行。但是，如果一个描述符只定义了_\_get__()方法，
则它的绑定关系比一般情况下要弱化很多（much weaker binding）。特别是，只有当被
访问的属性不在底层的实例字典中时，_\_get__()方法才会得到调用。


示例中的 lazyproperty 类通过让__get__()方法以 property 属性相同的名称来保存计算出
的值。这么做会让值保存在实例字典中，可以阻止该 property 属性重复进行计算。仔细
观察下面的示例就能发现这一点：

In [ ]:
c = Circle(4.0) 
print(vars(c))
print(c.area)
print(vars(c))
print(c.area )  # Notice access doesn't invoke property anymore

del c.area 
print(vars(c))

print(c.area)

本节讨论的技术有一个潜在的缺点，即，计算出的值在创建之后就变成可变的（mutable）
了。示例如下：

In [ ]:
print(c.area)
c.area=25
print(c.area)

如果需要考虑可变性的问题，可以使用另外一种方式实现，但执行效率会稍打折扣：

In [2]:
def lazyproperty(func):
    name = '_lazy_' + func.__name__
    @property
    def lazy(self):
        if hasattr(self, name):  #hasattr()函数用于判断对象是否包含对应的属性
            return getattr(self, name)
        else:
            value = func(self) #我这里认为是回调他自己,func指的是area函数,self指Circle对象c,这两个连用也就是执行area(self)
            setattr(self, name, value) #写入value
            return value
    return lazy

import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2 
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius 

c = Circle(4.0) 
print(c.area )
print(c.area )

<class '__main__.Circle'>
Computing area
50.26548245743669
50.26548245743669


如果使用这个版本的实现，就会发现 set 操作是不允许执行的。示例如下

In [2]:
c = Circle(4.0) 
print(c.area )
print(c.area )
# c.area = 25  # AttributeError: can't set attribute
# print(c.area )

Computing area
50.26548245743669
50.26548245743669


但是，这种方式的缺点就是所有的 get 操作都必须经由属性的 getter 函数来处理。这比
直接在实例字典中查找相应的值要慢一些。